# 模型评估

Model Evaluation

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics as skmet
import matplotlib.pyplot as plt
import seaborn as sb
import pprint
from feature_engine.encoding import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

## 评估措施分类

evaluation measures classification

In [ ]:
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.width', 75)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = '{:,.2f}'.format

nls97compba = pd.read_csv("data/nls97compba.csv")

feature_cols = ['satverbal','satmath','gpaoverall',
  'parentincome','gender']

# separate NLS data into train and test datasets
X_train, X_test, y_train, y_test =  \
  train_test_split(nls97compba[feature_cols],\
  nls97compba[['completedba']], test_size=0.3, random_state=0)
      
# do one hot encoding and scaling
ohe = OneHotEncoder(drop_last=True, variables=['gender'])
ohe.fit(X_train)
X_train_enc, X_test_enc = \
  ohe.transform(X_train), ohe.transform(X_test)

scaler = StandardScaler()
standcols = X_train_enc.iloc[:,:-1].columns
scaler.fit(X_train_enc[standcols])
X_train_enc = \
  pd.DataFrame(scaler.transform(X_train_enc[standcols]),
  columns=standcols, index=X_train_enc.index).\
  join(X_train_enc[['gender_Female']], how="left", on=None, validate="many_to_many")
X_test_enc = \
  pd.DataFrame(scaler.transform(X_test_enc[standcols]),
  columns=standcols, index=X_test_enc.index).\
  join(X_test_enc[['gender_Female']], how="left", on=None, validate="many_to_many")


# k nearest neighbor classification
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train_enc, y_train.values.ravel())
pred = knn.predict(X_test_enc)

# show a confusion matrix
cm = skmet.confusion_matrix(y_test, pred, labels=knn.classes_)
cmplot = skmet.ConfusionMatrixDisplay(confusion_matrix=cm, \
 display_labels=['Negative--No BA', 'Positive--Completed BA'])
cmplot.plot()
cmplot.ax_.set(title='Confusion Matrix', 
  xlabel='Predicted Value', ylabel='Actual Value')

# calculate accuracy, sensitivity, specificity, and precision
tn, fp, fn, tp = skmet.confusion_matrix(y_test.values.ravel(), pred).ravel()
tn, fp, fn, tp
accuracy = (tp + tn) / pred.shape[0]
accuracy
sensitivity = tp / (tp + fn)
sensitivity
specificity = tn / (tn+fp)
specificity
precision = tp / (tp + fp)
precision

print("accuracy: %.2f, sensitivity: %.2f, specificity: %.2f, precision: %.2f"  %
  (skmet.accuracy_score(y_test.values.ravel(), pred),
  skmet.recall_score(y_test.values.ravel(), pred),
  skmet.recall_score(y_test.values.ravel(), pred, pos_label=0),
  skmet.precision_score(y_test.values.ravel(), pred)))


# try this with a random forest classifier
rfc = RandomForestClassifier(n_estimators=100, max_depth=2, 
  n_jobs=-1, random_state=0)
rfc.fit(X_train_enc, y_train.values.ravel())
pred = rfc.predict(X_test_enc)
tn, fp, fn, tp = skmet.confusion_matrix(y_test.values.ravel(), pred).ravel()
tn, fp, fn, tp

print("accuracy: %.2f, sensitivity: %.2f, specificity: %.2f, precision: %.2f"  %
  (skmet.accuracy_score(y_test.values.ravel(), pred),
  skmet.recall_score(y_test.values.ravel(), pred),
  skmet.recall_score(y_test.values.ravel(), pred, pos_label=0),
  skmet.precision_score(y_test.values.ravel(), pred)))

## CAP/ROC曲线

CAP ROC curves

In [ ]:
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.width', 75)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = '{:,.3f}'.format

# a playing cards example
numobs = 6
inclasscnt = 3
plt.yticks([1,2,3])
plt.plot([0, numobs], [0, inclasscnt], c = 'b', label = 'Random Model')
plt.plot([0, inclasscnt, numobs], [0, inclasscnt, inclasscnt], c = 'grey', linewidth = 2, label = 'Perfect Model')
plt.title("Cumulative Accuracy Profile")
plt.xlabel("Total Cards")
plt.ylabel("In-class (Red) Cards")
plt.legend()

nls97compba = pd.read_csv("data/nls97compba.csv")

feature_cols = ['satverbal','satmath','gpaoverall',
  'parentincome','gender']

# separate NLS data into train and test datasets
X_train, X_test, y_train, y_test =  \
  train_test_split(nls97compba[feature_cols],\
  nls97compba[['completedba']], test_size=0.3, random_state=0)
      
# do one hot encoding and scaling
ohe = OneHotEncoder(drop_last=True, variables=['gender'])
ohe.fit(X_train)
X_train_enc, X_test_enc = \
  ohe.transform(X_train), ohe.transform(X_test)

scaler = StandardScaler()
standcols = X_train_enc.iloc[:,:-1].columns
scaler.fit(X_train_enc[standcols])
X_train_enc = \
  pd.DataFrame(scaler.transform(X_train_enc[standcols]),
  columns=standcols, index=X_train_enc.index).\
  join(X_train_enc[['gender_Female']], how="left", on=None, validate="many_to_many")
X_test_enc = \
  pd.DataFrame(scaler.transform(X_test_enc[standcols]),
  columns=standcols, index=X_test_enc.index).\
  join(X_test_enc[['gender_Female']], how="left", on=None, validate="many_to_many")


# random forest classification for feature importance
knn = KNeighborsClassifier(n_neighbors = 5)
rfc = RandomForestClassifier(n_estimators=100, max_depth=2, 
  n_jobs=-1, random_state=0)

# plot random model and perfect model
numobs = y_test.shape[0]
inclasscnt = y_test.iloc[:,0].sum()

plt.plot([0, numobs], [0, inclasscnt], c = 'b', label = 'Random Model')
plt.plot([0, inclasscnt, numobs], [0, inclasscnt, inclasscnt], c = 'grey', linewidth = 2, label = 'Perfect Model')
plt.axvline(numobs/2, color='black', linestyle='dashed', linewidth=1)
plt.axhline(numobs/2, color='black', linestyle='dashed', linewidth=1)
plt.title("Cumulative Accuracy Profile")
plt.xlabel("Total Observations")
plt.ylabel("In-class Observations")
plt.legend()

# plot k nearest neighbor and random forest models
def addplot(model, X, Xtest, y, modelname, linecolor):
  model.fit(X, y.values.ravel())
  probs = model.predict_proba(Xtest)[:, 1]
  
  probdf = pd.DataFrame(zip(probs, y_test.values.ravel()),
    columns=(['prob','inclass']))
  probdf.loc[-1] = [0,0]
  probdf = probdf.sort_values(['prob','inclass'],
    ascending=False).\
    assign(inclasscum = lambda x: x.inclass.cumsum())
  inclassmidpoint = probdf.iloc[int(probdf.shape[0]/2)].\
    inclasscum
  plt.axhline(inclassmidpoint, color=linecolor,
    linestyle='dashed', linewidth=1)
  plt.plot(np.arange(0, probdf.shape[0]),
    probdf.inclasscum, c = linecolor,
    label = modelname, linewidth = 4)

addplot(knn, X_train_enc, X_test_enc, y_train,
  'KNN', 'red')
addplot(rfc, X_train_enc, X_test_enc, y_train,
  'Random Forest', 'green')
plt.legend()


# plot probability distribution
rfc.fit(X_train_enc, y_train.values.ravel())
pred = rfc.predict(X_test_enc)
pred_probs = rfc.predict_proba(X_test_enc)[:, 1]

probdf = pd.DataFrame(zip(pred_probs, pred, y_test.values.ravel()),
  columns=(['prob','pred','actual']))

probdf.groupby(['pred'])['prob'].agg(['min','max'])

sb.kdeplot(probdf.loc[probdf.actual==1].prob, shade=True, color='red',
  label="Completed BA")
sb.kdeplot(probdf.loc[probdf.actual==0].prob, shade=True, color='green',
  label="Did Not Complete")
plt.axvline(0.5, color='black', linestyle='dashed', linewidth=1)
plt.axvline(0.65, color='black', linestyle='dashed', linewidth=1)
plt.title("Predicted Probability Distribution")
plt.legend(loc="upper left")


# plot ROC curve
fpr, tpr, ths = skmet.roc_curve(y_test, pred_probs)
ths = ths[1:]
fpr = fpr[1:]
tpr = tpr[1:]
fig, ax = plt.subplots()
ax.plot(ths, fpr, label="False Positive Rate")
ax.plot(ths, tpr, label="Sensitivity")
ax.set_title('False Positive Rate and Sensitivity by Threshold')
ax.set_xlabel('Threshold')
ax.set_ylabel('False Positive Rate and Sensitivity')
ax.legend()


fig, ax = plt.subplots()
ax.plot(fpr, tpr, linewidth=4, color="black")
ax.set_title('ROC curve')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('Sensitivity')

tholdind = np.where((ths>0.499) & (ths<0.501))[0][0]
tholdindlow = np.where((ths>0.397) & (ths<0.404))[0][0]
tholdindhigh = np.where((ths>0.599) & (ths<0.601))[0][0]
plt.vlines((fpr[tholdindlow],fpr[tholdind],fpr[tholdindhigh]),
  0, 1, linestyles ="dashed", colors =["green","blue","purple"])
plt.hlines((tpr[tholdindlow],tpr[tholdind],tpr[tholdindhigh]),
  0, 1, linestyles ="dashed", colors =["green","blue","purple"])


# plot precision and sensitivity lines
prec, sens, ths = skmet.precision_recall_curve(y_test, pred_probs)

prec = prec[1:-10]
sens = sens[1:-10]
ths  = ths[:-10]

fig, ax = plt.subplots()
ax.plot(ths, prec, label='Precision')
ax.plot(ths, sens, label='Sensitivity')
ax.set_title('Precision and Sensitivity by Threshold')
ax.set_xlabel('Threshold')
ax.set_ylabel('Precision and Sensitivity')
ax.set_xlim(0.3,0.9)
ax.legend()


# plot precision and sensitivity curve
fig, ax = plt.subplots()
ax.plot(sens, prec)
ax.set_title('Precision-Sensitivity Curve')
ax.set_xlabel('Sensitivity')
ax.set_ylabel('Precision')
ax.set_ylim(0)
plt.yticks(np.arange(0.2, 0.9, 0.2))

temp = pd.DataFrame(zip(sens, prec, ths), columns=['sensitivity','precision','threshold'])
temp.describe()
temp.head(70)
temp.tail(70)

## 多级评价

evaluation multiclass

In [ ]:
pd.set_option('display.width', 75)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = '{:,.0f}'.format

nls97degreelevel = pd.read_csv("data/nls97degreelevel.csv")

feature_cols = ['satverbal','satmath','gpaoverall',
  'parentincome','gender']

# separate NLS data into train and test datasets
X_train, X_test, y_train, y_test =  \
  train_test_split(nls97degreelevel[feature_cols],\
  nls97degreelevel[['degreelevel']], test_size=0.3, random_state=0)
      
# do one hot encoding and scaling
ohe = OneHotEncoder(drop_last=True, variables=['gender'])
ohe.fit(X_train)
X_train_enc, X_test_enc = \
  ohe.transform(X_train), ohe.transform(X_test)

scaler = StandardScaler()
standcols = X_train_enc.iloc[:,:-1].columns
scaler.fit(X_train_enc[standcols])
X_train_enc = \
  pd.DataFrame(scaler.transform(X_train_enc[standcols]),
  columns=standcols, index=X_train_enc.index).\
  join(X_train_enc[['gender_Female']], how="left", on=None, validate="many_to_many")
X_test_enc = \
  pd.DataFrame(scaler.transform(X_test_enc[standcols]),
  columns=standcols, index=X_test_enc.index).\
  join(X_test_enc[['gender_Female']], how="left", on=None, validate="many_to_many")


# k nearest neighbor classification model
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(X_train_enc, y_train.values.ravel())
pred = knn.predict(X_test_enc)
pred_probs = knn.predict_proba(X_test_enc)[:, 1]

# create a confusion matrix
cm = skmet.confusion_matrix(y_test, pred)
cmplot = skmet.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['High School', 'Bachelor','Post-Graduate'])
cmplot.plot()
cmplot.ax_.set(title='Confusion Matrix', 
  xlabel='Predicted Value', ylabel='Actual Value')

print(skmet.classification_report(y_test, pred,
  target_names=['High School', 'Bachelor','Post-Graduate']))

fpr, tpr, ths = skmet.roc_curve(y_test, pred_probs)
fig, ax = plt.subplots()
ax.plot(fpr, tpr, linewidth=4, color="black")
ax.set_title('ROC curve')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('Sensitivity')

nls97compba.dtypes
nls97 = pd.read_csv("data/nls97f.csv")
nls97.dtypes

nls97.highestdegree.value_counts().sort_index()

nls97['degreelevel'] = \
  np.where(nls97.highestdegree.isnull(),np.nan,
  np.where(nls97.highestdegree.str[0:1].isin(['2','3']),2,
  np.where(nls97.highestdegree.str[0:1].isin(['4']),3,
  np.where(nls97.highestdegree.str[0:1].isin(['5','6','7']),4,1))))

nls97.groupby(['degreelevel','highestdegree']).size().reset_index()

nls97.degreelevel.value_counts()

nls97degreelevel = pd.merge(nls97compba, nls97[['personid','degreelevel']], left_on=['personid'], right_on=['personid'], how="left")
nls97degreelevel = nls97degreelevel.loc[nls97degreelevel.degreelevel>1]
nls97degreelevel['degreelevel'] = nls97degreelevel.degreelevel-1
nls97degreelevel.groupby(['degreelevel','completedba']).size().reset_index()

nls97degreelevel.to_csv("data/nls97degreelevel.csv")

# plot random model and perfect model
numobs = y_test.shape[0]
inclasscnt = y_test.iloc[:,0].sum()

plt.plot([0, numobs], [0, inclasscnt], c = 'b', label = 'Random Model')
plt.plot([0, inclasscnt, numobs], [0, inclasscnt, inclasscnt], c = 'grey', linewidth = 2, label = 'Perfect Model')
plt.axvline(numobs/2, color='black', linestyle='dashed', linewidth=1)
plt.axhline(numobs/2, color='black', linestyle='dashed', linewidth=1)
plt.title("Cumulative Accuracy Profile")
plt.xlabel("Total Observations")
plt.ylabel("In-class Observations")
plt.legend()

# plot k nearest neighbor and random forest models
def addplot(model, X, Xtest, y, modelname, linecolor):
  model.fit(X, y.values.ravel())
  probs = model.predict_proba(Xtest)[:, 1]
  
  probdf = pd.DataFrame(zip(probs, y_test.values.ravel()),
    columns=(['prob','inclass']))
  probdf.loc[-1] = [0,0]
  probdf = probdf.sort_values(['prob','inclass'],
    ascending=False).\
    assign(inclasscum = lambda x: x.inclass.cumsum())
  inclassmidpoint = probdf.iloc[int(probdf.shape[0]/2)].\
    inclasscum
  plt.axhline(inclassmidpoint, color=linecolor,
    linestyle='dashed', linewidth=1)
  plt.plot(np.arange(0, probdf.shape[0]),
    probdf.inclasscum, c = linecolor,
    label = modelname, linewidth = 4)

addplot(knn, X_train_enc, X_test_enc, y_train,
  'KNN', 'red')
addplot(rfc, X_train_enc, X_test_enc, y_train,
  'Random Forest', 'green')
plt.legend()

# plot probability distribution
knn.fit(X_train_enc, y_train.values.ravel())
pred = knn.predict(X_test_enc)
cm = skmet.confusion_matrix(y_test, pred)
cm

# plot ROC curve
fpr, tpr, ths = skmet.roc_curve(y_test, pred_probs)
ths = ths[1:]
fpr = fpr[1:]
tpr = tpr[1:]
fig, ax = plt.subplots()
ax.plot(ths, fpr, label="False Positive Rate")
ax.plot(ths, tpr, label="Sensitivity")
ax.set_title('False Positive Rate and Sensitivity by Threshold')
ax.set_xlabel('Threshold')
ax.set_ylabel('False Positive Rate and Sensitivity')
ax.legend()


fig, ax = plt.subplots()
ax.plot(fpr, tpr, linewidth=4, color="black")
ax.set_title('ROC curve')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('Sensitivity')

tholdind = np.where((ths>0.499) & (ths<0.501))[0][0]
tholdindlow = np.where((ths>0.397) & (ths<0.404))[0][0]
tholdindhigh = np.where((ths>0.599) & (ths<0.601))[0][0]
plt.vlines((fpr[tholdindlow],fpr[tholdind],fpr[tholdindhigh]),
  0, 1, linestyles ="dashed", colors =["green","blue","purple"])
plt.hlines((tpr[tholdindlow],tpr[tholdind],tpr[tholdindhigh]),
  0, 1, linestyles ="dashed", colors =["green","blue","purple"])


# plot precision and sensitivity lines
sens, prec, ths = skmet.precision_recall_curve(y_test, pred_probs)
sens = sens[1:-10]
prec = prec[1:-10]
ths  = ths[:-10]

fig, ax = plt.subplots()
ax.plot(ths, prec, label='Precision')
ax.plot(ths, sens, label='Sensitivity')
ax.set_title('Precision and Sensitivity by Threshold')
ax.set_xlabel('Threshold')
ax.set_ylabel('Precision and Sensitivity')
ax.set_xlim(0.3,0.9)
ax.legend()

# plot precision and sensitivity curve
fig, ax = plt.subplots()
ax.plot(sens, prec)
ax.set_title('Precision-Sensitivity Curve')
ax.set_xlabel('Sensitivity')
ax.set_ylabel('Precision')
ax.set_xlim(0.5,0.9)

temp = pd.DataFrame(zip(sens, prec, ths), columns=['sensitivity','precision','threshold'])
temp.describe()
temp.head(70)
temp.tail(70)

## 回归度量

regression metrics

In [ ]:
from sklearn.linear_model import LinearRegression

pd.set_option('display.width', 150)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:,.3f}'.format

# load the NLS data
landtemps = pd.read_csv("data/landtemps2019avgs.csv")

feature_cols = ['latabs','elevation']

X_train, X_test, y_train, y_test =  \
  train_test_split(landtemps[feature_cols],\
  landtemps[['avgtemp']], test_size=0.3, random_state=0)
      
# scale the data  
scaler = StandardScaler()
scaler.fit(X_train)
X_train = \
  pd.DataFrame(scaler.transform(X_train),
  columns=feature_cols, index=X_train.index)
X_test = \
  pd.DataFrame(scaler.transform(X_test),
  columns=feature_cols, index=X_test.index)

scaler.fit(y_train)
y_train, y_test = \
  pd.DataFrame(scaler.transform(y_train),
  columns=['avgtemp'], index=y_train.index),\
  pd.DataFrame(scaler.transform(y_test),
  columns=['avgtemp'], index=y_test.index)


# use linear regression for recursive feature elimination
lr = LinearRegression()
lr.fit(X_train, y_train)
np.column_stack((lr.coef_.ravel(),X_test.columns.values))

# get predictions and residuals
pred = lr.predict(X_test)

preddf = pd.DataFrame(pred, columns=['prediction'],
  index=X_test.index).join(X_test, how="left", on=None, validate="many_to_many").join(y_test)
preddf['resid'] = preddf.avgtemp-preddf.prediction

preddf.resid.agg(['mean','median','skew','kurtosis'])

# plot the residuals
plt.hist(preddf.resid, color="blue")
plt.axvline(preddf.resid.mean(), color='red', linestyle='dashed', linewidth=1)
plt.title("Histogram of Residuals for Temperature Model")
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.show()

# plot predictions against the residuals
plt.scatter(preddf.prediction, preddf.resid, color="blue")
plt.axhline(0, color='red', linestyle='dashed', linewidth=1)
plt.title("Scatterplot of Predictions and Residuals")
plt.xlabel("Predicted Temperature")
plt.ylabel("Residuals")
plt.show()

# generate summary model evaluation statistics
mse = skmet.mean_squared_error(y_test, pred)
mse
rmse = skmet.mean_squared_error(y_test, pred, squared=False)
rmse
mae = skmet.mean_absolute_error(y_test, pred)
mae
r2 = skmet.r2_score(y_test, pred)
r2
   
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)
pred = knn.predict(X_test)

mae = skmet.mean_absolute_error(y_test, pred)
mae
r2 = skmet.r2_score(y_test, pred)
r2

preddf = pd.DataFrame(pred, columns=['prediction'],
  index=X_test.index).join(X_test).join(y_test, how="left", on=None, validate="many_to_many")
preddf['resid'] = preddf.avgtemp-preddf.prediction

# plot predictions against the residuals
plt.scatter(preddf.prediction, preddf.resid, color="blue")
plt.axhline(0, color='red', linestyle='dashed', linewidth=1)
plt.title("Scatterplot of Predictions and Residuals with KNN Model")
plt.xlabel("Predicted Temperature")
plt.ylabel("Residuals")
plt.show()

## 交叉验证

cross validation

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

pd.set_option('display.width', 150)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:,.3f}'.format

# load the NLS data
landtemps = pd.read_csv("data/landtemps2019avgs.csv")

feature_cols = ['latabs','elevation']

X_train, X_test, y_train, y_test =  \
  train_test_split(landtemps[feature_cols],\
  landtemps[['avgtemp']], test_size=0.1, random_state=0)

kf = KFold(n_splits=5, shuffle=True, random_state=0)
      
# use linear regression for recursive feature elimination
def getscores(model):
  pipeline = make_pipeline(StandardScaler(), model)
  scores = cross_validate(pipeline, X=X_train, y=y_train,
    cv=kf, scoring=['r2'], n_jobs=1)
  scorelist.append(dict(model=str(model),
    fit_time=scores['fit_time'].mean(),
    r2=scores['test_r2'].mean()))


scorelist = []
getscores(LinearRegression())
getscores(RandomForestRegressor(max_depth=2))
getscores(KNeighborsRegressor(n_neighbors=5))

scorelist

pipeline = make_pipeline(StandardScaler(), LinearRegression())
scores = cross_validate(pipeline, X=X_train, y=y_train,
  cv=kf, scoring=['r2'], n_jobs=1)
scores

## 网格搜索

grid search

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
pd.set_option('display.width', 150)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:,.3f}'.format

# load the NLS data
landtemps = pd.read_csv("data/landtemps2019avgs.csv")

feature_cols = ['latabs','elevation']

X_train, X_test, y_train, y_test =  \
  train_test_split(landtemps[feature_cols],\
  landtemps[['avgtemp']], test_size=0.3, random_state=0)
      
# X = landtemps[feature_cols]
# y = landtemps[['avgtemp']]

# use linear regression for recursive feature elimination
lr = LinearRegression()
split = KFold(n_splits=5, shuffle=True, random_state=0)

pipe = Pipeline([
 ('scaler', StandardScaler()),
 ('lr', LinearRegression())
        ])

pipeline = make_pipeline(StandardScaler(), LinearRegression())
scores = cross_validate(pipeline, X=X_train, y=y_train, cv=split, n_jobs=1, return_estimator = True)
scores
type(scores)
temp = pipeline.fit(X_train, y_train)

# get predictions and residuals
pred = lr.predict(X_test)

preddf = pd.DataFrame(pred, columns=['prediction'],
  index=X_test.index).join(X_test).join(y_test)
preddf['resid'] = preddf.avgtemp-preddf.prediction

preddf.resid.agg(['mean','median','skew','kurtosis'])

# generate summary model evaluation statistics
mse = skmet.mean_squared_error(y_test, pred)
mse
rmse = skmet.mean_squared_error(y_test, pred, squared=False)
rmse
mae = skmet.mean_absolute_error(y_test, pred)
mae
r2 = skmet.r2_score(y_test, pred)
r2

## 管道预处理

pipeline preprocessing

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from feature_engine.encoding import OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import cross_validate, KFold
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor

import os
import sys
sys.path.append(os.getcwd() + "/helperfunctions")
from preprocfunc import OutlierTrans


pd.set_option('display.width', 200)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = '{:,.0f}'.format

# load the NLS data and create training and testing DataFrames
nls97wages = pd.read_csv("data/nls97wagesb.csv")
nls97wages.set_index("personid", inplace=True)

nls97wages.dropna(subset=['wageincome'], inplace=True)
nls97wages.loc[nls97wages.motherhighgrade==95,
  'motherhighgrade'] = np.nan
nls97wages.loc[nls97wages.fatherhighgrade==95,
  'fatherhighgrade'] = np.nan

# setup the features and target
num_cols = ['gpascience','gpaenglish','gpamath','gpaoverall',
  'motherhighgrade','fatherhighgrade','parentincome']
cat_cols = ['gender']
bin_cols = ['completedba']

nls97wages[['wageincome'] + num_cols].agg(['count','min','median','max']).T

target = nls97wages[['wageincome']]
features = nls97wages[num_cols + cat_cols + bin_cols]

X_train, X_test, y_train, y_test =  \
  train_test_split(features,\
  target, test_size=0.2, random_state=0)


# setup pipelines for column transformation
standtrans = make_pipeline(OutlierTrans(2),
  StandardScaler())
cattrans = make_pipeline(SimpleImputer(strategy="most_frequent"),
  OneHotEncoder(drop_last=True))
bintrans = make_pipeline(SimpleImputer(strategy="most_frequent"))

coltrans = ColumnTransformer(
  transformers=[
    ("stand", standtrans, num_cols),
    ("cat", cattrans, ['gender']),
    ("bin", bintrans, ['completedba'])
  ]
)

# add feature selection and a linear model to the pipeline and look at the parameter estimates
lr = LinearRegression()

pipe1 = make_pipeline(coltrans,
  KNNImputer(n_neighbors=5), lr)

ttr=TransformedTargetRegressor(regressor=pipe1,
  transformer=StandardScaler())


# run kfold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=0)

scores = cross_validate(ttr, X=X_train, y=y_train,
  cv=kf, scoring=('r2', 'neg_mean_absolute_error'),
  n_jobs=1)

print("Mean Absolute Error: %.2f, R-squared: %.2f" % 
  (scores['test_neg_mean_absolute_error'].mean(),
  scores['test_r2'].mean()))

Mean Absolute Error: -23755.52, R-squared: 0.20
